In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas beautifulsoup4 html5lib

Note: you may need to restart the kernel to use updated packages.


In [1]:
from tkinter import ttk
from tkinter import *
import tkinter as tk
import pymysql
from tkinter import messagebox,filedialog
from flask import Flask, session, redirect, url_for
import pandas as pd
import tabula
import pdfplumber
import os
import html5lib
import bs4
import numpy as np
import re
import pyodbc
import sqlite3
import time

root=tk.Tk()
root.title('DashBoard')
root.geometry('1270x668+0+0')
root.resizable(1,1)
root.config(bg='beige')

bg_Image=PhotoImage(file='inventory_logo.png')
titleLabel=Label(root,image=bg_Image,compound=LEFT,text=' Inventory Management System',font=('times new roman',40,'bold'),bg='#010c48',fg='white',anchor='w',padx=20)
titleLabel.place(x=0,y=0,relwidth=1)

def logout():
    answer = messagebox.askyesno("Logout", "Are you sure you want to logout?")
    if answer:
        messagebox.showinfo("Logged Out", "You have been logged out.")
        root.destroy()
logoutButton=Button(root,text='LogOut',font=('times new roman',20,'bold'),fg='#010c48',command=logout)
logoutButton.place(x=1100,y=10)

subtitleLabel=Label(root,text='Welcome Admin\t\t Date: 08-07-2024\t\t Time: 12:36:17 pm',font=('times new roman',15),bg='#4d636d',fg='white')
subtitleLabel.place(x=0,y=70,relwidth=1)
def update_datetime():
    current_time = time.strftime("%I:%M:%S %p")  # 12-hour format time with AM/PM
    current_date = time.strftime("%d-%m-%Y")
    subtitleLabel.config(text=f"Welcome Admin\t\t Date: {current_date}\t\t Time: {current_time}")
    root.after(1000, update_datetime)  # call this function again after 1000ms (1 second)

update_datetime()  # initial call to start the clock

leftFrame=Frame(root)
leftFrame.place(x=0,y=102,width=200,height=667)

logo_img=PhotoImage(file='checklist.png')
imageLabel=Label(leftFrame,image=logo_img)
imageLabel.pack()

menuLabel=Label(leftFrame,text='Menu',font=('times new roman',20),bg='#009688')
menuLabel.pack(fill=X)

client_icon=PhotoImage(file='client.png')
client_button=Button(leftFrame,image=client_icon,compound=LEFT,text='Clients',font=('times new roman',20,'bold'))
client_button.pack(fill=X)

#prod,panel,inv,bill,sales,exit
product_icon=PhotoImage(file='product.png')
product_button=Button(leftFrame,image=product_icon,compound=LEFT,text='Invoice',font=('times new roman',20,'bold'),command=lambda:product_form(root))
product_button.pack(fill=X)


def product_form(root):
    global back_image, df, file_name, file_path

    product_window = tk.Toplevel()
    product_window.title("Invoice")
    product_window.attributes("-fullscreen", True)
    product_window.grab_set()

    product_frame = Frame(product_window, bg='white')
    product_frame.place(relx=0, rely=0, relwidth=1, relheight=1)

    top_close_button = Button(product_window, text='X', font=('Arial', 14, 'bold'),
                              bg='red', fg='white', cursor='hand2', bd=0,
                              command=product_window.destroy)
    top_close_button.place(relx=1.0, x=-15, y=20, anchor='ne', width=40, height=40)

    heading_label = Label(product_frame, text='Get Inventory Files',
                          font=('times new roman', 18, 'bold'),
                          bg='#0f4d7d', fg='white', anchor='w')
    heading_label.place(x=20, y=20, relwidth=0.95, height=40)

    search_frame = LabelFrame(product_frame, text='Search Product',
                              font=('times new roman', 14), padx=10, pady=10)
    search_frame.place(relx=0.05, rely=0.12, relwidth=0.9, height=80)

    for i in range(4):
        search_frame.columnconfigure(i, weight=1)

    search_combobox = ttk.Combobox(search_frame,
                                   values=('HTML', 'Excel', 'CSV', 'TXT', 'PDF'),
                                   state='readonly', font=('times new roman', 14))
    search_combobox.grid(row=0, column=0, padx=5, pady=10, sticky='ew')
    search_combobox.set("Select File Type")

    output_frame = Frame(product_frame, bg='white')
    output_frame.place(relx=0.05, rely=0.25, relwidth=0.9, relheight=0.68)

    output_text = Text(output_frame, bg='white', font=('Consolas', 12), wrap='none')
    output_text.pack(side=LEFT, fill=BOTH, expand=True)
    heading_label=Label(output_text,text='Result Of Searched Files',font=('times new roman',16,'bold'),bg='#0f4d7d',fg='white')
    heading_label.place(x=0,y=0,relwidth=1)

    scroll_y = Scrollbar(output_frame, orient=VERTICAL, command=output_text.yview)
    scroll_y.pack(side=RIGHT, fill=Y)

    scroll_x = Scrollbar(output_frame, orient=HORIZONTAL, command=output_text.xview)
    scroll_x.pack(side=BOTTOM, fill=X)

    output_text.config(yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set)

 #   output_text.insert('1.0', 'Result Of Searched Files' + ' ' * 220 + '\n')

  #  output_text.tag_add('heading', '1.0', '1.end')
   # output_text.tag_config('heading', font=('times new roman', 16, 'bold'),
    #                       background='#0f4d7d', foreground='white')
  #  output_text.tag_configure("center", justify='center')
   # output_text.tag_add("center", "1.0", "1.end")

    file_types = {
        "HTML": (".html", pd.read_html),
        "Excel": (".xlsx", pd.read_excel),
        "CSV": (".csv", pd.read_csv),
        "TXT": (".txt", pd.read_csv),
        "PDF": (".pdf", None),
    }

    def pdf_to_excel(file_path, output_excel_path):
        with pdfplumber.open(file_path) as pdf:
            all_tables = []
            for page in pdf.pages:
                tables = page.extract_tables()
                for table in tables:
                    if table and len(table) > 1:
                        df = pd.DataFrame(table[1:], columns=table[0])
                        all_tables.append(df)
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])
                except Exception:
                    # Fallback if header row is malformed
                    df = pd.DataFrame(table)
                all_tables.append(df)

    
    def analyze_pdf_bill(file_path):
        global text
        with pdfplumber.open(file_path) as pdf:
            all_text = ""
            tables = []
            
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    print(f"\n--- Page {page_num + 1} Text ---")
                    print(text)
                    all_text += text + "\n"
                
                # Extract any tables found on the page
                page_tables = page.extract_tables()
                for table in page_tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # Use first row as header
                    print(f"\n--- Page {page_num + 1} Table ---")
                    print(df)
                    tables.append(df)
                    
            if not tables:
                print(f"No tables found on page {page_num + 1}")
        
            return all_text, tables

    # Example usage
  #  pdf_file = "path/to/your/bill.pdf"
   # text, tables = analyze_pdf_bill(pdf_file)


    def extract_metadata(text):
        invoice_no = re.search(r"Invoice\s*No[:\-]?\s*(\w+)", text)
        total_amt = re.search(r"Total\s*Amount[:\-]?\s*(\d+\.\d{2})", text)
        return {
            "invoice_no": invoice_no.group(1) if invoice_no else None,
            "total_amt": total_amt.group(1) if total_amt else None,
        }
    
    # Example usage:
#    metadata = extract_metadata(text)
#    print("Invoice No:", metadata['invoice_no'])
 #   print("Total Amount:", metadata['total_amt'])

    def clean_dataframe(df):
        try:
            df.dropna(how='all', inplace=True)
            df.fillna(value=0, inplace=True)
            for col in df.select_dtypes(include=['object']):
                df[col] = df[col].str.strip()
            for col in df.columns:
                try:
                    df[col] = pd.to_numeric(df[col])
                except:
                    continue
            df.drop_duplicates(inplace=True)
            return df
        except Exception as e:
            print(f"An error occurred during cleaning: {e}")
            return None
           

    def load_file(event=None):
        global df, file_name, file_path
        file_type = search_combobox.get()
        if file_type not in file_types:
            messagebox.showerror("Error", "Unsupported file type selected.")
            return

        ext, reader = file_types[file_type]
        file_path = filedialog.askopenfilename(title="Select a file", filetypes=[(f"{file_type} files", f"*{ext}")])
        if not file_path:
            messagebox.showwarning("No file", "No file was selected.")
            return

        file_name = os.path.basename(file_path)
        entry_var.set(file_name)

        try:
            if file_type == "HTML":
                df_list = reader(file_path)
                if not df_list:
                    raise ValueError("No tables found in HTML.")
                df = df_list[0]
            elif file_type == "PDF":
                output_excel = "output_file.xlsx"
                pdf_to_excel(file_path, output_excel)
                df = pd.read_excel(output_excel)
            else:
                df = reader(file_path)

            df = clean_dataframe(df)

            output_text.delete("1.0", tk.END)
            output_text.insert(tk.END, f"Loaded {file_name} ({len(df)} rows):\n\n")
            output_text.insert(tk.END, df.head().to_string(index=False))

        except Exception as e:
            messagebox.showerror("Read Error", f"Could not read file:\n{e}")
            
    def add_file_to_sql():
        if df is None or df.empty:
            messagebox.showwarning("No Data", "Please load a valid file with data first.")
            return
    
        try:
            conn = pyodbc.connect(
                "DRIVER={ODBC Driver 17 for SQL Server};"
                "SERVER=LAPTOP-AQMG9SG5\\SQLEXPRESS;"
                "DATABASE=files;"
                "Trusted_Connection=yes;"
            )
            cursor = conn.cursor()
    
            # Create table if not exists
            cursor.execute("""
                IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='NEWTAB' AND xtype='U')
                CREATE TABLE NEWTAB(
                   cat_num INT PRIMARY KEY IDENTITY(1,1),
                   make VARCHAR(50),
                   material_name VARCHAR(50),
                   quantity DECIMAL(10,2),
                   discount DECIMAL(10,2),
                   rate_per_pannel DECIMAL(10,2),
                   total DECIMAL(10,2)
                );
            """)
            conn.commit()
    
            inserted = 0
            updated = 0
    
            for _, row in df.iterrows():
                try:
                    make = str(row.get('make', '')).strip()
                    name = str(row.get('material_name', '')).strip()
                    qty = Decimal(row.get('quantity', 0))
                    discount = Decimal(row.get('discount', 0))
                    rate = Decimal(row.get('rate_per_pannel', 0))
    
                    # Check if a matching row already exists
                    cursor.execute("""
                        SELECT quantity, discount FROM NEWTAB
                        WHERE make = ? AND material_name = ? AND rate_per_pannel = ?
                    """, (make, name, rate))
                    existing = cursor.fetchone()
    
                    if existing:
                        # Row exists, update by adding new quantity and discount
                        existing_qty, existing_discount = existing
                        new_qty = existing_qty + qty
                        new_discount = existing_discount + discount
                        new_total = round(new_qty * rate, 2)
    
                        cursor.execute("""
                            UPDATE NEWTAB
                            SET quantity = ?, discount = ?, total = ?
                            WHERE make = ? AND material_name = ? AND rate_per_pannel = ?
                        """, (new_qty, new_discount, new_total, make, name, rate))
                        updated += 1
                    else:
                        # Insert new row
                        total = round(qty * rate, 2)
                        cursor.execute("""
                            INSERT INTO NEWTAB (make, material_name, quantity, discount, rate_per_pannel, total)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (make, name, qty, discount, rate, total))
                        inserted += 1
    
                except Exception as row_e:
                    print(f"Row processing failed: {row_e}")
    
            conn.commit()
            messagebox.showinfo("Success", f"{inserted} row(s) inserted, {updated} row(s) updated in SQL.")
    
        except Exception as e:
            messagebox.showerror("Database Error", f"Failed to insert/update data:\n{e}")
    
        finally:
            if 'conn' in locals():
                conn.close()

    search_combobox.bind("<<ComboboxSelected>>", load_file)
    entry_var = tk.StringVar()
    file_id_var = StringVar()
    search_entry = Entry(search_frame, font=('times new roman', 14, 'bold'),
                         bg='lightyellow', textvariable=entry_var)
    search_entry.grid(row=0, column=1, padx=5, sticky='ew')

    search_button = Button(search_frame, text='Add', font=('times new roman', 14),
                           cursor='hand2', fg='white', bg='#0f4d7d', command=lambda:  add_file_to_sql())
    search_button.grid(row=0, column=2, padx=5, sticky='ew')



def inventory_form(root):
    global back_image
    inventory_window = tk.Toplevel()
    inventory_window.title("Inventory")

    # Make it full screen
    inventory_window.attributes("-fullscreen", True)

    # Make it modal (block other windows)
    inventory_window.grab_set()

    # Optional: add a button to close the window
    exit_button = ttk.Button(inventory_window, text="Close", command=inventory_window.destroy)
    exit_button.pack(pady=10)

    # Inventory Frame (full width and height)
    inventory_frame = Frame(inventory_window, bg='white')
    inventory_frame.place(relx=0, rely=0, relwidth=1, relheight=1)
    
    # Heading label
    heading_label = Label(inventory_frame, text='Manage Inventory Details',
                          font=('times new roman', 18, 'bold'),
                          bg='#0f4d7d', fg='white', anchor='w')
    heading_label.place(x=20, y=20, relwidth=0.95, height=40)
    
    conn = pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=LAPTOP-AQMG9SG5\\SQLEXPRESS;"
        "DATABASE=files;"
        "Trusted_Connection=yes;"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT TOP 1 * FROM NEWTAB")
    print(cursor.fetchone())
    conn.close()
    
    def create_inventory_table():
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LAPTOP-AQMG9SG5\\SQLEXPRESS;"
            "DATABASE=files;"
            "Trusted_Connection=yes;"
        )
        cursor = conn.cursor()
    
        cursor.execute("""
            IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='NEWTAB' AND xtype='U')
            CREATE TABLE NEWTAB (
               cat_num INT PRIMARY KEY IDENTITY(1,1),
               make VARCHAR(50),
               material_name VARCHAR(50),
               quantity DECIMAL(10,2),
               discount DECIMAL(10,2),
               rate_per_pannel DECIMAL(10,2),
               total DECIMAL(10,2)
            );
        """)
        conn.commit()
        print("✅ NEWTAB table created successfully")
    
        conn.close()
    
    # Call it directly
    create_inventory_table()
    
        # Utility Functions
    def clean_dataframe(df):
        try:
            df.dropna(how='all', inplace=True)
            df.fillna(value=0, inplace=True)
            for col in df.select_dtypes(include=['object']):
                df[col] = df[col].str.strip()
            df.drop_duplicates(inplace=True)
            return df
        except Exception as e:
            print(f"An error occurred during cleaning: {e}")
            return None 
    
    def merge_and_update_inventory(df_new, conn_str):
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        df_existing = pd.read_sql("SELECT * FROM NEWTAB", conn)
    
        for _, new_row in df_new.iterrows():
            make = new_row['make']
            material_name = new_row['material_name']
            rate_per_pannel = new_row['rate_per_pannel']
            new_qty = new_row['quantity']
            new_discount = new_row['discount']
    
            # Find a matching row
            match = df_existing[
                (df_existing['make'] == make) &
                (df_existing['material_name'] == material_name) &
                (df_existing['rate_per_pannel'] == rate_per_pannel)
            ]
    
            if not match.empty:
                # Match found - update quantity and discount
                existing_row = match.iloc[0]
                new_total_qty = existing_row['quantity'] + new_qty
                new_total_discount = existing_row['discount'] + new_discount
    
                update_query = """
                    UPDATE NEWTAB
                    SET quantity = ?, discount = ?
                    WHERE id = ?
                """
                cursor.execute(update_query, new_total_qty, new_total_discount, existing_row['id'])
    
            else:
                # No match - insert new row
                insert_query = """
                    INSERT INTO NEWTAB (make, material_name, rate_per_pannel, quantity, discount)
                    VALUES (?, ?, ?, ?, ?)
                """
                cursor.execute(insert_query, make, material_name, rate_per_pannel, new_qty, new_discount)
    
        conn.commit()
        cursor.close()
        conn.close()
        print("Inventory merge and update complete.")

    def load_inventory_to_treeview(treeview):
        conn_str = (
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LAPTOP-AQMG9SG5\\SQLEXPRESS;"
            "DATABASE=files;"
            "Trusted_Connection=yes;"
        )
    
        conn = pyodbc.connect(conn_str)
        columns = ["cat_num", "make", "material_name", "quantity", "discount", "rate_per_pannel", "total"]
        df = pd.read_sql("SELECT * FROM NEWTAB", conn) 
    
        # Clear Treeview
        for item in treeview.get_children():
            treeview.delete(item)
    
        # Configure columns
        treeview["columns"] = columns
        treeview["show"] = "headings"
        for col in columns:
            treeview.heading(col, text=col)
            treeview.column(col, anchor="center", width=150)
    
        # Insert rows
        for _, row in df.iterrows():
            treeview.insert('', 'end', values=list(row))
    
        conn.close()

        
  
    def search_inventory():
        field = search_combobox.get()
        value = search_entry.get().strip()
    
        if field not in ('Make', 'Material name') or not value:
            print("Please select a valid field and enter search text.")
            return None
    
        column = 'make' if field == 'Make' else 'material_name'
    
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LAPTOP-AQMG9SG5\\SQLEXPRESS;"
            "DATABASE=files;"
            "Trusted_Connection=yes;"
        )
        query = f"""
            SELECT cat_num ,make, material_name, quantity, discount, rate_per_pannel, total
            FROM NEWTAB
            WHERE {column} LIKE ?
        """
        df_results = pd.read_sql_query(query, conn, params=(value + '%',))
        conn.close()
    
        # Update Treeview
        treeview.delete(*treeview.get_children())
        treeview["columns"] = df_results.columns.tolist()
        treeview["show"] = "headings"
    
        for col in df_results.columns:
            treeview.heading(col, text=col)
            treeview.column(col, anchor='center')
    
        for _, row in df_results.iterrows():
            treeview.insert('', 'end', values=list(row))
    
        # Return DataFrame if needed later
        return df_results

        # Search Frame
    search_frame = LabelFrame(inventory_frame, text='Search Product',
                              font=('times new roman', 14), padx=10, pady=10)
    search_frame.place(relx=0.05, rely=0.12, relwidth=0.9, height=80)

     # Grid configuration for uniform spacing
    for i in range(5):
        search_frame.columnconfigure(i, weight=1)
    
    search_combobox = ttk.Combobox(search_frame, values=('Make', 'Material name'),
                                    state='readonly', width=16, font=('times new roman', 14))
    search_combobox.grid(row=0, column=0, padx=5, sticky='ew')
    search_combobox.set('Search By')
    
    search_entry = Entry(search_frame, font=('times new roman', 14, 'bold'),
                         bg='lightyellow')
    search_entry.grid(row=0, column=1, padx=5, sticky='ew')
    
    search_button = Button(search_frame, text='Search', font=('times new roman', 14),
                           cursor='hand2', fg='white', bg='#0f4d7d', command=search_inventory)
    search_button.grid(row=0, column=2, padx=5, sticky='ew')
    
    show_button = Button(search_frame, text='Show All', font=('times new roman', 14),
                         cursor='hand2', fg='white', bg='#0f4d7d',
                         command=lambda: load_inventory_to_treeview(treeview))
    show_button.grid(row=0, column=3, padx=5, sticky='ew')

    def calculate_total_column():
        for item in treeview.get_children():
            values = list(treeview.item(item, 'values'))
            try:
                quantity = float(values[3])
                rate = float(values[5])
                total = round(quantity * rate, 2)
            except (ValueError, IndexError):
                total = 'Error'
            
            # Extend or replace the 'total' value at index 6
            if len(values) < 7:
                values.append(total)
            else:
                values[6] = total
    
            treeview.item(item, values=values)
            
    # Close button at top-right
    top_close_button = Button(inventory_window, text='X', font=('Arial', 14, 'bold'),
                              bg='red', fg='white', cursor='hand2', bd=0,
                              command=inventory_window.destroy)
    top_close_button.place(relx=1.0, x=-15, y=19, anchor='ne', width=40, height=40)

    
    total_button = Button(search_frame, text='Total', font=('times new roman', 14),
                          cursor='hand2', fg='white', bg='#0f4d7d', command=calculate_total_column)
    total_button.grid(row=0, column=4, padx=5, sticky='ew')
    
    # Treeview Frame
    treeview_frame = Frame(inventory_frame, bg='grey')
    treeview_frame.place(relx=0.05, rely=0.25, relwidth=0.9, relheight=0.65)
    
    # Treeview + Scrollbars
    scrolly = Scrollbar(treeview_frame, orient=VERTICAL)
    scrollx = Scrollbar(treeview_frame, orient=HORIZONTAL)
    
    treeview = ttk.Treeview(treeview_frame,
                            columns=('cat_num', 'make', 'material_name', 'quantity',
                                     'discount', 'rate_per_pannel', 'total'),
                            show='headings',
                            yscrollcommand=scrolly.set,
                            xscrollcommand=scrollx.set)
    
    scrolly.pack(side=RIGHT, fill=Y)
    scrollx.pack(side=BOTTOM, fill=X)
    treeview.pack(fill=BOTH, expand=1)
    
    scrollx.config(command=treeview.xview)
    scrolly.config(command=treeview.yview)
    
    # Headings
    treeview.heading('cat_num', text='Cat Num')
    treeview.heading('make', text='Make')
    treeview.heading('material_name', text='Material Name')
    treeview.heading('quantity', text='Quantity')
    treeview.heading('discount', text='Discount')
    treeview.heading('rate_per_pannel', text='Rate per Panel')
    treeview.heading('total', text='Total')
    
    # Optionally: adjust column widths
    for col in treeview['columns']:
        treeview.column(col, anchor='center', width=150)


inventory_icon=PhotoImage(file='products.png')
inventory_button=Button(leftFrame,image=inventory_icon,compound=LEFT,text='Inventory',font=('times new roman',20,'bold'),anchor='w',padx=1,command=lambda:inventory_form(root))
inventory_button.pack(fill=X)

billing_icon=PhotoImage(file='bill.png')
billing_button=Button(leftFrame,image=billing_icon,compound=LEFT,text='Billings',font=('times new roman',20,'bold'))
billing_button.pack(fill=X)

sales_icon=PhotoImage(file='sales.png')
sales_button=Button(leftFrame,image=sales_icon,compound=LEFT,text='Saless',font=('times new roman',20,'bold'))
sales_button.pack(fill=X)

def exit_app():
    answer = messagebox.askyesno("Confirm Exit", "Are you sure you want to exit?")
    if answer:  
        root.destroy() 
logout_icon=PhotoImage(file='logout.png')
logout_button=Button(leftFrame,image=logout_icon,compound=LEFT,text='Exit',font=('times new roman',20,'bold'),command=exit_app)
logout_button.pack(fill=X)


#total_client
emp_frame=Frame(root,bg='#2C3E50',bd=3,relief=RIDGE)
emp_frame.place(x=400,y=125,height=170,width=280)
total_client_icon=PhotoImage(file='total_client.png')
total_client_icon_label=Label(emp_frame,image=total_client_icon,bg='#2C3E50')
total_client_icon_label.pack()

total_client_icon_label=Label(emp_frame,text='Total Clients',bg='#2C3E50',fg='white',font=('times new roman',15,'bold'))
total_client_icon_label.pack()

total_client_count_label=Label(emp_frame,text='0',bg='#2C3E50',fg='white',font=('times new roman',15,'bold'))
total_client_count_label.pack()
#total_Supply
supply_frame=Frame(root,bg='#8E44AD',bd=3,relief=RIDGE)
supply_frame.place(x=800,y=125,height=170,width=280)
total_supply_icon=PhotoImage(file='total_supply.png')
total_supply_icon_label=Label(supply_frame,image=total_supply_icon,bg='#8E44AD')
total_supply_icon_label.pack()

total_supply_label=Label(supply_frame,text='Total Supply',bg='#8E44AD',fg='white',font=('times new roman',15,'bold'))
total_supply_label.pack()

total_supply_count_label=Label(supply_frame,text='0',bg='#8E44AD',fg='white',font=('times new roman',15,'bold'))
total_supply_count_label.pack()
#total revenue
revenue_frame=Frame(root,bg='#8E44AD',bd=3,relief=RIDGE)
revenue_frame.place(x=400,y=310,height=170,width=280)
total_revenue_icon=PhotoImage(file='total_rev.png')
total_revenue_icon_label=Label(revenue_frame,image=total_revenue_icon,bg='#8E44AD')
total_revenue_icon_label.pack()

total_revenue_label=Label(revenue_frame,text='Total Revenue',bg='#8E44AD',fg='white',font=('times new roman',15,'bold'))
total_revenue_label.pack()

total_revenue_count_label=Label(revenue_frame,text='0',bg='#8E44AD',fg='white',font=('times new roman',15,'bold'))
total_revenue_count_label.pack()
# total product
product_frame=Frame(root,bg='#2C3E50',bd=3,relief=RIDGE)
product_frame.place(x=800,y=310,height=170,width=280)
total_product_icon=PhotoImage(file='total_product.png')
total_product_icon_label=Label(product_frame,image=total_product_icon,bg='#2C3E50')
total_product_icon_label.pack()

total_product_label=Label(product_frame,text='Total Product',bg='#2C3E50',fg='white',font=('times new roman',15,'bold'))
total_product_label.pack()

total_product_count_label=Label(product_frame,text='0',bg='#2C3E50',fg='white',font=('times new roman',15,'bold'))
total_product_count_label.pack()
#total order
order_frame=Frame(root,bg='#27AE60',bd=3,relief=RIDGE)
order_frame.place(x=600,y=490,height=170,width=280)
total_order_icon=PhotoImage(file='total_order.png')
total_order_icon_label=Label(order_frame,image=total_order_icon,bg='#27AE60')
total_order_icon_label.pack()

total_order_label=Label(order_frame,text='Total Order',bg='#27AE60',fg='white',font=('times new roman',15,'bold'))
total_order_label.pack()

total_order_count_label=Label(order_frame,text='0',bg='#27AE60',fg='white',font=('times new roman',15,'bold'))
total_order_count_label.pack()

root.mainloop()

(1, 'pop', 'steel', Decimal('25.00'), Decimal('0.00'), Decimal('1000.00'), Decimal('25000.00'))
✅ NEWTAB table created successfully


C:\Users\GAURANG\AppData\Local\Temp\ipykernel_29640\2234419033.py:478: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM NEWTAB", conn)
